In [23]:
!pip install transformers datasets evaluate rouge_score pandas scikit-learn

In [24]:
import pandas as pd
from datasets import Dataset
from transformers import RobertaTokenizer, RobertaForQuestionAnswering, pipeline
from rouge_score import rouge_scorer
import nltk
from nltk.translate.meteor_score import meteor_score

# Ensure NLTK resources are downloaded
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [25]:

# Load CSV files
train_df = pd.read_csv('https://raw.githubusercontent.com/viswa3024/aiml-capstone-project-email/main/question-answering/Dataset-2/train.csv')
dev_df = pd.read_csv('https://raw.githubusercontent.com/viswa3024/aiml-capstone-project-email/main/question-answering/Dataset-2/dev.csv')
test_df = pd.read_csv('https://raw.githubusercontent.com/viswa3024/aiml-capstone-project-email/main/question-answering/Dataset-2/test.csv')

# Convert DataFrames to Datasets
train_dataset = Dataset.from_pandas(train_df)
dev_dataset = Dataset.from_pandas(dev_df)
test_dataset = Dataset.from_pandas(test_df)

In [26]:
print(train_dataset[0])
print(dev_dataset[0])
print(test_dataset[0])

{'question': 'What is the role of Siamese networks in domain adaptation?', 'answer': 'Siamese networks can be used to align the representations of source and target domains by minimizing the discrepancy between them, making the model more robust to domain shifts.'}
{'question': 'How do U-Nets skip connections contribute to better segmentation performance?', 'answer': 'Skip connections in U-Net preserve fine-grained spatial information and improving localization and detail in segmentation tasks.'}
{'question': 'What is feature selection?', 'answer': 'The process of selecting a subset of relevant features to improve model performance and reduce complexity.'}


In [27]:
import pandas as pd
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

In [28]:
# Load tokenizer and model
model_name = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [29]:
# Tokenize the dataset
def preprocess_function(examples):
    inputs = examples["question"]
    targets = examples["answer"]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [30]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
dev_dataset = dev_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1985 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

Map:   0%|          | 0/249 [00:00<?, ? examples/s]

In [31]:
import evaluate

In [32]:
rouge_metric = evaluate.load("rouge")

In [33]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)

In [34]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=15,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",  # Set to "tensorboard" if using TensorBoard
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    #compute_metrics=compute_metrics
)

# Fine-tune the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.724800,1.156235
2,0.857400,0.776807
3,0.792000,0.738668
4,0.791900,0.720193
5,0.704800,0.708045
6,0.678600,0.698937
7,0.720100,0.692256
8,0.650100,0.687420
9,0.691600,0.683206
10,0.645600,0.680359


TrainOutput(global_step=3735, training_loss=1.1263667563677153, metrics={'train_runtime': 918.0686, 'train_samples_per_second': 32.432, 'train_steps_per_second': 4.068, 'total_flos': 1383722444390400.0, 'train_loss': 1.1263667563677153, 'epoch': 15.0})

In [35]:
import numpy as np

In [36]:
import json
import pandas as pd

def save_predictions_and_labels(test_dataset, trainer, file_prefix=''):
    predictions = []
    labels = []

    for batch in trainer.get_test_dataloader(test_dataset):
        outputs = trainer.model(**batch)
        logits = outputs.logits
        pred_ids = np.argmax(logits.detach().cpu().numpy(), axis=-1)

        batch_preds = pred_ids.tolist()
        batch_labels = batch['labels'].tolist()

        predictions.extend(batch_preds)
        labels.extend(batch_labels)

        # Save periodically to manage memory
        if len(predictions) % 1000 == 0:
            pd.DataFrame({'predictions': [json.dumps(p) for p in predictions], 'labels': [json.dumps(l) for l in labels]}).to_csv(f'{file_prefix}_predictions_labels.csv', index=False)

    # Final save
    pd.DataFrame({'predictions': [json.dumps(p) for p in predictions], 'labels': [json.dumps(l) for l in labels]}).to_csv(f'{file_prefix}_predictions_labels.csv', index=False)

save_predictions_and_labels(test_dataset, trainer, 'test')


In [37]:
import pandas as pd
import json
import evaluate
from transformers import T5Tokenizer



# Load ROUGE metric
rouge_metric = evaluate.load("rouge")

def compute_rouge_from_file(file_path):
    df = pd.read_csv(file_path)
    # Convert JSON strings back to lists
    decoded_preds = [json.loads(p) for p in df['predictions']]
    decoded_labels = [json.loads(l) for l in df['labels']]

    # Decode text
    decoded_preds_text = tokenizer.batch_decode(decoded_preds, skip_special_tokens=True)
    decoded_labels_text = tokenizer.batch_decode(decoded_labels, skip_special_tokens=True)

    return rouge_metric.compute(predictions=decoded_preds_text, references=decoded_labels_text)

# Compute ROUGE scores
rouge_scores = compute_rouge_from_file('test_predictions_labels.csv')

print("ROUGE Scores:")
print(rouge_scores)


ROUGE Scores:
{'rouge1': 0.44836666333837344, 'rouge2': 0.18548711386397476, 'rougeL': 0.41570182791205057, 'rougeLsum': 0.41611921607976643}


In [38]:
# Generate predictions
def generate_predictions(test_dataset, trainer):
    predictions = []
    for batch in trainer.get_test_dataloader(test_dataset):
        outputs = trainer.model.generate(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
        decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        predictions.extend(decoded_preds)
    return predictions

# Get final output
final_output = generate_predictions(test_dataset, trainer)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [39]:
test_df["Generated"] = final_output
test_df.to_csv('google_flan_t5_small_qna_dataset_2_v1.csv')

In [40]:
df_generated = pd.read_csv('google_flan_t5_small_qna_dataset_2_v1.csv')

In [41]:
df_generated.head(10)

,Unnamed: 0,question,answer,Generated
0,0,What is feature selection?,The process of selecting a subset of relevant ...,A technique used to select features that are n...
1,1,"What are Markov decision processes (MDP), and ...",MDPs are mathematical frameworks for modeling ...,MDP is a type of decision processing that invo...
2,2,What is batch normalization and how does it work?,Normalizes the layer's input over a mini-batch,Batch normalization is a technique used to nor...
3,3,How does Word2Vec create word embeddings?,Word2Vec creates word embeddings by training o...,Word2Vec creates word embeddings using a vecto...
4,4,What is the role of hidden layers in MLP?,Hidden layers in MLP help capture complex patt...,Hidden layers are hidden layers that are used ...
5,5,What is a Decision Tree?,A Decision Tree is a predictive learning algor...,A Decision Tree is a type of tree that consist...
6,6,What is variance in machine learning?,Variance refers to the error introduced by the...,Variance in machine learning involves averagin...
7,7,What are common methods to diagnose bias and v...,Common methods include analyzing learning curv...,"Common methods include a regression model, a r..."
8,8,What is the difference between L1 and L2 regul...,L1 regularization adds the absolute values of ...,L1 regularization is used to regularize the in...
9,9,What is feature engineering for unstructured d...,Feature engineering for unstructured data invo...,Feature engineering for unstructured data invo...


In [42]:
model.save_pretrained("google_flan_t5_small_qna_dataset_2_v1")
tokenizer.save_pretrained("google_flan_t5_small_qna_dataset_2_v1")

('google_flan_t5_small_qna_dataset_2_v1/tokenizer_config.json',
 'google_flan_t5_small_qna_dataset_2_v1/special_tokens_map.json',
 'google_flan_t5_small_qna_dataset_2_v1/spiece.model',
 'google_flan_t5_small_qna_dataset_2_v1/added_tokens.json')

In [43]:
from huggingface_hub import notebook_login

In [44]:
notebook_login()

In [45]:
repo_name="google_flan_t5_small_qna_dataset_2_v1"

model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kkasiviswanath/google_flan_t5_small_qna_dataset_2_v1/commit/8b67c3751c2dc8f88a0c2e58be80df531a497a43', commit_message='Upload tokenizer', commit_description='', oid='8b67c3751c2dc8f88a0c2e58be80df531a497a43', pr_url=None, pr_revision=None, pr_num=None)

In [46]:
from transformers import pipeline

In [47]:

model_name = 'kkasiviswanath/google_flan_t5_small_qna_dataset_2_v1'

# Load the model and tokenizer using the pipeline
qa_pipeline = pipeline('text2text-generation', model=model_name, tokenizer=model_name)

# Define a question
question = "Can we utilize an autoencoder to perform dimensionality reduction on numerical datasets?"

# Generate a prediction
result = qa_pipeline(question)

# Print the result
print(result)

config.json:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'Autoencoders can be used to reduce the dimensionality of numerical datasets by'}]


In [48]:
# Define a question
question = "How does Word2Vec create word embeddings?"

# Generate a prediction
result = qa_pipeline(question)

# Print the result
print(result)

[{'generated_text': 'Word2Vec creates word embeddings using a vector based on'}]


In [49]:
from transformers import pipeline

# Define the model path
model_name = 'kkasiviswanath/google_flan_t5_small_qna_dataset_2_v1'

# Load the QA pipeline with your model
qa_pipeline = pipeline('text2text-generation', model=model_name, tokenizer=model_name)

# Define your question
question = "How does Word2Vec create word embeddings"

# Generate the answer with adjusted parameters
result = qa_pipeline(question, max_length=100, num_beams=5, temperature=1.0, top_p=0.9)

# Output the result
print(f"Question: {question}")
print(f"Answer: {result[0]['generated_text']}")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Question: How does Word2Vec create word embeddings
Answer: Word2Vec creates word embeddings using Word2Vec.


In [50]:
# Define your question
question = "What is the role of hidden layers in MLP?"

# Generate the answer with adjusted parameters
result = qa_pipeline(question, max_length=100, num_beams=5, temperature=1.0, top_p=0.9)

# Output the result
print(f"Question: {question}")
print(f"Answer: {result[0]['generated_text']}")

Question: What is the role of hidden layers in MLP?
Answer: Hidden layers are used in MLP to capture hidden data, allowing the model to learn more complex features.
